In [1]:
from utils import *
import torch
from models import ExtractDialogueModel
from transcribe import whisper_transcribe
from utils import set_openai_key

In [2]:
openai_key_path = 'openai_api_key.txt'
# openai_key_path = 'personal_openai_api_key.txt'

device = torch.device("cuda")
params_extract_dialogue = {
    'speaker_diarization_model': 'pyannote/speaker-diarization-3.1',
    'speaker_embedding_model': 'pyannote/embedding',
    'hf_token_path': 'huggingface_token.txt',
    'openai_key_path': openai_key_path, 
    'transcribe_fn': whisper_transcribe,
    'full_audio_path': '../../full_audios/LFB9_full.wav',
    'interval': 180,
    'console_times_path': '../../annotations/console_times/combined_console_times_secs.csv',
    'fb_annot_path': '../../clips_no_wiggle/fbk_cuts_no_wiggle_0_4210.csv',
    'vad_activity_path': '../../full_VADs/LFB1_full_activity.csv',
    'diarizations_save_path': 'results/extract_dialogue/diarizations/LFB9_full.csv',
    'transcriptions_save_path': 'results/extract_dialogue/transcriptions/LFB9_full.csv',
    'identifications_save_path': 'results/extract_dialogue/identifications/LFB9_full.csv',
    'fb_detection_save_path': "results/extract_dialogue/fb_detection/LFB9_full 'all phrases'.csv",
    'audio_clips_dir': 'results/extract_dialogue/audio_clips',
    'trainer_anchors_dir': 'results/extract_dialogue/anchors/trainer',
    'trainee_anchors_dir': 'results/extract_dialogue/anchors/trainee',
    'tmp_dir': 'tmp',
    'seed': 42,
    'min_n_speakers': 2,
    'max_n_speakers': 2,
    'embedding_dist_thresh': 0.8
}
set_openai_key(openai_key_path)

In [3]:
model = ExtractDialogueModel(params_extract_dialogue, device)

Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`
Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.4.0+cu118. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.4.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [4]:
model.full_diarization(load_saved=True)
model.full_transcription(load_saved=True)
model.full_identification(load_saved=True)
model.full_fb_detection(load_saved=True)
model.full_aligned_fb_detection(load_saved=True)
behavior_pred = model.full_behavior_prediction(load_saved=True)

In [10]:
t, p = model.evaluate(weighting='binary', model_type='fb')

In [27]:
export = pd.read_csv("results/extract_dialogue/aligned_fb_detection/LFB9_full 'all phrases'.csv")
export[['pred_fb_instance', 'true_fb_instance']]
true = export['true_fb_instance']
pred = export['pred_fb_instance']

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

precision_recall_fscore_support(true, pred, average='binary')

(0.6753246753246753, 0.5, 0.574585635359116, None)

In [28]:
len(export)

310

In [24]:
true.value_counts(), pred.value_counts()

(true_fb_instance
 False    102
 True      73
 Name: count, dtype: int64,
 pred_fb_instance
 False    94
 True     81
 Name: count, dtype: int64)

In [15]:
t.value_counts(), p.value_counts()

(pred_fb_instance
 0    233
 1     77
 Name: count, dtype: int64,
 true_fb_instance
 0    206
 1    104
 Name: count, dtype: int64)

In [6]:
model.evaluate(weighting='binary', model_type='behavior')

{'precision_r_t_verb': 0.5454545454545454,
 'recall_r_t_verb': 0.8571428571428571,
 'f1_r_t_verb': 0.6666666666666666,
 'accuracy_r_t_verb': 0.5384615384615384,
 'roc_auc_r_t_verb': 0.5119047619047619,
 'precision_r_t_beh': 0.7878787878787878,
 'recall_r_t_beh': 0.7027027027027027,
 'f1_r_t_beh': 0.7428571428571429,
 'accuracy_r_t_beh': 0.6538461538461539,
 'roc_auc_r_t_beh': 0.618018018018018,
 'precision_r_t_clarify': 0.0,
 'recall_r_t_clarify': 0.0,
 'f1_r_t_clarify': 0.0,
 'accuracy_r_t_clarify': 0.9807692307692307,
 'roc_auc_r_t_clarify': None}

In [7]:
model.evaluate(weighting='weighted', model_type='behavior')

{'precision_r_t_verb': 0.5244755244755245,
 'recall_r_t_verb': 0.5384615384615384,
 'f1_r_t_verb': 0.47435897435897434,
 'accuracy_r_t_verb': 0.5384615384615384,
 'roc_auc_r_t_verb': 0.5119047619047619,
 'precision_r_t_beh': 0.6820635504846031,
 'recall_r_t_beh': 0.6538461538461539,
 'f1_r_t_beh': 0.6643180349062702,
 'accuracy_r_t_beh': 0.6538461538461539,
 'roc_auc_r_t_beh': 0.618018018018018,
 'precision_r_t_clarify': 1.0,
 'recall_r_t_clarify': 0.9807692307692307,
 'f1_r_t_clarify': 0.9902912621359223,
 'accuracy_r_t_clarify': 0.9807692307692307,
 'roc_auc_r_t_clarify': None}

In [8]:
model.evaluate(weighting='weighted', model_type='behavior')

{'precision_r_t_verb': 0.5244755244755245,
 'recall_r_t_verb': 0.5384615384615384,
 'f1_r_t_verb': 0.47435897435897434,
 'accuracy_r_t_verb': 0.5384615384615384,
 'roc_auc_r_t_verb': 0.5119047619047619,
 'precision_r_t_beh': 0.6820635504846031,
 'recall_r_t_beh': 0.6538461538461539,
 'f1_r_t_beh': 0.6643180349062702,
 'accuracy_r_t_beh': 0.6538461538461539,
 'roc_auc_r_t_beh': 0.618018018018018,
 'precision_r_t_clarify': 1.0,
 'recall_r_t_clarify': 0.9807692307692307,
 'f1_r_t_clarify': 0.9902912621359223,
 'accuracy_r_t_clarify': 0.9807692307692307,
 'roc_auc_r_t_clarify': None}

In [9]:
# def hms2secs(hms):
#     h, m, s = hms.split(':')
#     return int(h) * 3600 + int(m) * 60 + int(s)

# def update_times(old_fb_annots, new_fb_annots):
#     df = old_fb_annots.copy()
    
#     for case in new_fb_annots['Case'].unique():
#         case_id = int(case.replace('LFB', ''))
        
#         old = old_fb_annots[old_fb_annots['Case'] == case_id]
#         new = new_fb_annots[new_fb_annots['Case'] == case]
        
#         if len(old) != len(new):
#             continue
        
#         offset = hms2secs(old.iloc[0]['Timestamp']) - hms2secs(old.iloc[0]['fbk_time'])

#         for i in range(len(old)):
#             old_timestamp = old.iloc[i]['Timestamp']
#             new_timestamp = new.iloc[i]['FB Timestamp VALUES']
            
#             if old_timestamp != new_timestamp:
#                 diff = hms2secs(new_timestamp) - hms2secs(old_timestamp)
#                 print(case_id, old_timestamp, new_timestamp)
                
# pd.set_option('display.max_colwidth', None)
# old_fb_annots = pd.read_csv('../../clips_no_wiggle/fbk_cuts_no_wiggle_0_4210.csv', index_col=0)
# new_fb_annots = pd.read_csv('../../annotations/LFB Delay All Cases.csv', index_col=0)
# update_times(old_fb_annots, new_fb_annots)